In [61]:
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv("All-baca-7040x73.csv")


In [62]:
from numpy.random import RandomState
rng = RandomState()
df = pd.read_csv("All-baca-7040x73.csv")
train = df.sample(frac=0.7, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

In [63]:
train['KATA'].value_counts()

HAUS      624
TOILET    624
LAPAR     620
MAKAN     619
SENANG    618
MINUM     613
SEDIH     608
SAKIT     602
Name: KATA, dtype: int64

In [64]:
test['KATA'].value_counts()

SAKIT     278
SEDIH     272
MINUM     267
SENANG    262
MAKAN     261
LAPAR     260
HAUS      256
TOILET    256
Name: KATA, dtype: int64

In [67]:
# Seperating Predictors and Outcome values from train and test sets
X_train = pd.DataFrame(train.drop(['SUBJEK','KATA'],axis=1))


X_test = pd.DataFrame(test.drop(['SUBJEK','KATA'],axis=1))


In [68]:
X_train

,LABEL,AF3_THETA,AF3_ALPHA,AF3_LOW_BETA,AF3_HIGH_BETA,AF3_GAMMA,F7_THETA,F7_ALPHA,F7_LOW_BETA,F7_HIGH_BETA,...,F8_THETA,F8_ALPHA,F8_LOW_BETA,F8_HIGH_BETA,F8_GAMMA,AF4_THETA,AF4_ALPHA,AF4_LOW_BETA,AF4_HIGH_BETA,AF4_GAMMA
1332,17,0.969157,0.419813,0.250860,0.187390,0.065156,0.885038,0.578492,0.466285,0.239371,...,0.800284,0.288642,0.144675,0.206214,0.229648,0.085342,0.022524,0.020913,0.015468,0.018381
6298,7,0.553792,0.297000,0.282370,0.298493,0.129987,0.633558,0.486767,0.389702,0.455692,...,0.871469,0.469115,0.601600,0.269455,0.207066,1.218318,0.382762,0.806457,0.413249,0.250350
3276,22,14.952279,3.280859,0.529933,0.506541,0.332382,23.593871,1.705054,1.081661,1.232416,...,11.597608,2.486005,0.388443,1.563786,1.344028,12.899495,3.239416,0.455126,0.828596,0.464897
5290,12,1.564874,3.649904,0.206752,0.444108,0.245853,3.166249,2.514987,0.973730,0.419433,...,2.315902,2.599993,0.828052,0.294634,0.148104,1.685567,3.248761,1.279734,0.626797,0.236227
997,32,2.506063,1.075268,1.054692,0.957324,0.467257,1.159595,0.428635,0.463035,0.699421,...,2.426510,2.457515,1.414658,1.308107,0.385595,2.318869,1.075982,2.143163,2.837157,0.744531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6769,37,51.565604,3.391266,1.835776,0.808303,0.296889,19.835506,1.089665,0.737173,0.687855,...,2.642095,0.616125,0.234319,0.297825,0.266174,53.489877,4.621722,2.171978,0.809025,0.263999
5713,27,7.756195,2.240452,0.848055,0.881694,0.413748,7.974846,1.066862,0.665752,0.486006,...,7.430139,4.179854,2.339862,0.663899,0.335339,25.352437,6.233271,1.161865,0.844477,0.775728
4626,7,12.596277,2.305350,1.429092,2.475099,0.940114,2.770743,0.700636,0.681848,1.568421,...,2.919247,0.425634,0.571025,0.874110,0.416580,16.017680,1.519394,1.255895,2.342372,1.076702
5897,2,1.380722,0.367982,0.225017,0.254739,0.215541,2.050639,0.539228,0.201793,0.455422,...,0.557181,0.693332,0.579581,0.239018,0.158849,2.682565,0.769564,0.454318,0.324629,0.216097


In [69]:
X_test

,LABEL,AF3_THETA,AF3_ALPHA,AF3_LOW_BETA,AF3_HIGH_BETA,AF3_GAMMA,F7_THETA,F7_ALPHA,F7_LOW_BETA,F7_HIGH_BETA,...,F8_THETA,F8_ALPHA,F8_LOW_BETA,F8_HIGH_BETA,F8_GAMMA,AF4_THETA,AF4_ALPHA,AF4_LOW_BETA,AF4_HIGH_BETA,AF4_GAMMA
1,2,3.033470,2.028336,0.933330,6.659462,0.916482,1.824260,1.254074,0.634456,3.700502,...,10.334194,2.218516,1.273915,0.610819,0.342051,5.532071,2.337674,1.011535,1.609977,1.325849
4,2,6.828661,3.425307,1.405881,4.518988,0.532399,1.986457,1.560375,1.256536,3.083162,...,8.196053,3.192119,2.291647,0.731953,0.302914,11.453557,4.499370,1.867601,1.266913,0.623921
5,2,13.394172,4.604422,1.378888,3.345854,0.468798,3.194034,1.656977,1.265359,2.396279,...,8.333990,3.374497,2.260097,0.858818,0.315853,20.202041,6.215337,1.977753,1.299966,0.530632
11,2,42.181576,6.109872,0.419378,2.499464,0.982025,9.553105,1.213163,0.408726,1.556084,...,13.633903,2.008298,0.682593,0.877496,0.617408,57.783356,8.898400,0.909611,2.577918,0.928984
12,2,33.202433,4.308125,0.370781,2.277905,0.980550,7.778650,0.882705,0.421524,1.462749,...,11.934303,1.572880,0.739357,0.894378,0.590198,46.233436,6.497908,0.794796,2.854827,0.902377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7017,32,8.809801,2.691115,0.700470,0.355971,0.135588,4.754065,1.414664,0.733873,0.275685,...,3.156630,1.022644,0.371568,0.358461,0.276610,7.852246,2.320847,0.876808,0.645105,0.363037
7018,32,4.825134,1.969288,0.563214,0.358036,0.135428,3.344331,1.141357,0.805275,0.287915,...,3.010638,1.045846,0.456660,0.405954,0.314061,5.476381,1.840115,0.876011,0.666304,0.339792
7023,32,1.448134,0.569360,0.526131,0.369438,0.166980,1.372291,0.641029,0.686642,0.288760,...,0.929340,0.826974,0.409080,0.489482,0.361166,3.632138,0.880122,1.006690,0.999960,0.345115
7024,37,19.891139,3.686975,0.514632,0.372545,0.141639,4.749370,1.807512,0.189985,0.468722,...,0.764684,1.063684,0.459404,0.267332,0.233134,14.151417,2.361383,0.646940,0.517115,0.358956


In [ ]:
Label

In [25]:
Y_train_label

array([37, 17, 37, ..., 27, 27, 27], dtype=object)

In [23]:
Y_test_label

array([2, 2, 2, ..., 37, 37, 37], dtype=object)

In [29]:
# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
encoder.fit(Y_train_label)
Y_train = encoder.transform(Y_train_label)

# encoding test labels 
encoder.fit(Y_test_label)
Y_test = encoder.transform(Y_test_label)

#Total Number of Continous and Categorical features in the training set
num_cols = X_train._get_numeric_data().columns
print("Number of numeric features:",num_cols.size)
#list(set(X_train.columns) - set(num_cols))


names_of_predictors = list(X_train.columns.values)


Number of numeric features: 70


In [27]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from numpy.random import RandomState

In [30]:
final_model = SVC(kernel='rbf', gamma=0.001, C=1000)
#final_model = SVC(kernel='poly', gamma=0.001, C=1000)
final_model.fit(X_train, Y_train)

SVC(C=1000, gamma=0.001)

In [32]:
Y_pred = final_model.predict(X_test)
Y_pred_label = list(encoder.inverse_transform(Y_pred))

print("Training set score for SVM: %f" % final_model.score(X_train , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test  , Y_test ))

Training set score for SVM: 0.965300
Testing  set score for SVM: 0.786932


In [34]:
n_classes=8

In [44]:
Y_test

array([0, 0, 0, ..., 7, 7, 7])

In [48]:
Y_pred

array([0, 0, 0, ..., 7, 7, 7])

In [53]:
clf = OneVsRestClassifier(LinearSVC(random_state=0))
y_score = clf.fit(X_train, Y_train).decision_function(X_test)

C:\Users\IMBA PC\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\IMBA PC\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\IMBA PC\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\IMBA PC\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\IMBA PC\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of 

In [55]:
X_train

,AF3_THETA,AF3_ALPHA,AF3_LOW_BETA,AF3_HIGH_BETA,AF3_GAMMA,F7_THETA,F7_ALPHA,F7_LOW_BETA,F7_HIGH_BETA,F7_GAMMA,...,F8_THETA,F8_ALPHA,F8_LOW_BETA,F8_HIGH_BETA,F8_GAMMA,AF4_THETA,AF4_ALPHA,AF4_LOW_BETA,AF4_HIGH_BETA,AF4_GAMMA
6256,0.423073,0.849213,0.143045,0.177825,0.146152,0.776716,0.318053,0.677775,0.455227,0.897437,...,1.724391,1.065701,0.322749,0.323232,0.257977,1.150256,1.131571,0.472789,0.236763,0.204469
568,3.561036,1.248834,0.402016,1.685657,1.889830,1.820449,0.887768,0.247769,1.034733,0.597373,...,1.937794,0.583892,0.450815,0.585681,0.630813,2.793595,0.719009,0.339058,1.253548,1.712881
1531,1.095015,0.625003,0.433012,0.232953,0.076381,0.873730,0.619836,0.668202,0.249569,0.161224,...,2.086927,0.812313,0.349884,0.517981,0.114014,0.071300,0.030951,0.024047,0.015958,0.014983
4954,2.048056,2.531181,1.241439,2.332025,0.550160,0.826937,1.165857,0.552366,0.695373,0.536284,...,4.017442,1.339636,0.879380,2.063724,0.599968,1.997180,4.318983,2.228683,4.503985,0.933629
6145,0.916234,0.489746,0.654671,0.217297,0.156947,1.383343,0.501079,0.776464,0.749042,0.393293,...,1.652656,0.507012,1.010707,0.258741,0.274295,2.161136,0.407456,1.284984,0.320132,0.209659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5124,2.941636,1.434361,0.359110,0.426032,0.217967,2.681471,1.051165,0.658066,0.400820,0.183097,...,7.136089,3.492337,2.853636,0.721638,0.429971,4.094978,2.381994,0.998692,0.581087,0.328790
6763,35.507696,5.779094,2.805934,0.728369,0.142891,13.072971,1.055892,1.662943,0.816902,0.275374,...,2.497251,0.451089,0.469619,0.763812,0.211456,33.214026,3.304922,1.406709,0.775797,0.299266
3538,0.937816,1.144966,1.254642,2.179178,0.964822,0.988851,0.696238,0.353891,0.681386,0.315350,...,1.103930,1.342256,0.537233,0.681767,0.308752,0.990176,2.444644,0.678045,2.081431,1.372917
5968,1.025311,0.790290,0.761975,0.238270,0.184917,1.381843,1.439977,0.993738,0.588498,0.547966,...,1.478954,0.707603,0.890104,0.429664,0.389146,3.167747,1.073649,1.510128,0.559347,0.333705


In [52]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[i], Y_pred[i])
    roc_auc[i] = auc(fpr[i], tpr[i])

TypeError: Singleton array 0 cannot be considered a valid collection.

In [ ]:
# shuffle and split training and test sets
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.33, random_state=0)

# classifier
clf = OneVsRestClassifier(LinearSVC(random_state=0))
y_score = clf.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()